In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import pyspark.sql.functions as F
from pyspark.sql.functions import udf

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ShortType, ByteType, DateType

In [2]:
CREDENTIALS_LOCATION = '/home/emilel/.secrets/gcp/gcp-secret.json'
SPARK_LIB = "/home/emilel/dez-project-emi/lib/gcs-connector-hadoop3-2.2.5.jar"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", SPARK_LIB) \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", CREDENTIALS_LOCATION)

In [3]:
sc = SparkContext.getOrCreate(conf=conf)
sc.setLogLevel("WARN")
hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", CREDENTIALS_LOCATION)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/03/26 01:41:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
idm_schema = StructType([
    StructField('year', ShortType(), True)
    , StructField('entity_code', ByteType(), True)
    , StructField('entity_name', StringType(), True)
    , StructField('municipality_code', ShortType(), True)
    , StructField('municipality_name', StringType(), True)
    , StructField('affected_legal_asset', StringType(), True)
    , StructField('crime_type', StringType(), True)
    , StructField('crime_subtype', StringType(), True)
    , StructField('crime_modality_type', StringType(), True)
    , StructField('january', IntegerType(), True)
    , StructField('february', IntegerType(), True)
    , StructField('march', IntegerType(), True)
    , StructField('april', IntegerType(), True)
    , StructField('may', IntegerType(), True)
    , StructField('june', IntegerType(), True)
    , StructField('july', IntegerType(), True)
    , StructField('august', IntegerType(), True)
    , StructField('september', IntegerType(), True)
    , StructField('october', IntegerType(), True)
    , StructField('november', IntegerType(), True)
    , StructField('december', IntegerType(), True)
])

In [6]:
unpivoting_columns = ['year'
                      , 'entity_code'
                      , 'entity_name'
                      , 'municipality_code'
                      , 'municipality_name'
                      , 'affected_legal_asset'
                      , 'crime_type'
                      , 'crime_subtype'
                      , 'crime_modality_type']


stack_query_expression = '''
    STACK(12
    , "january", january
    , "february", february
    , "march", march
    , "april", april
    , "may", may
    , "june", june
    , "july", july
    , "august", august
    , "september", september
    , "october", october
    , "november", november
    , "december", december
    ) AS (month, crimes)
'''

month_dict = {
    'january'     : '01-01'
    , 'february'  : '02-01'
    , 'march'     : '03-01'
    , 'april'     : '04-01'
    , 'may'       : '05-01'
    , 'june'      : '06-01'
    , 'july'      : '07-01'
    , 'august'    : '08-01'
    , 'september' : '09-01'
    , 'october'   : '10-01'
    , 'november'  : '11-01'
    , 'december'  : '12-01'
}

In [7]:
@udf(returnType=StringType())
def get_first_day_of_month_date_udf(year, month_name):
    return str(year) + '-' + month_dict[month_name]

In [14]:
idm_df = spark.read\
    .option('header', True)\
    .schema(idm_schema)\
    .csv('gs://landing_bucket_dez/idm/idm.csv')
idm_df.show()

24/03/26 01:54:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: A�o, Clave_Ent, Entidad, Cve. Municipio, Municipio, Bien jur�dico afectado, Tipo de delito, Subtipo de delito, Modalidad, Enero, Febrero, Marzo, Abril, Mayo, Junio, Julio, Agosto, Septiembre, Octubre, Noviembre, Diciembre
 Schema: year, entity_code, entity_name, municipality_code, municipality_name, affected_legal_asset, crime_type, crime_subtype, crime_modality_type, january, february, march, april, may, june, july, august, september, october, november, december
Expected: year but found: A�o
CSV file: gs://landing_bucket_dez/idm/idm.csv
+----+-----------+--------------+-----------------+-----------------+--------------------+-----------+-----------------+--------------------+-------+--------+-----+-----+---+----+----+------+---------+-------+--------+--------+
|year|entity_code|   entity_name|municipality_code|municipality_name|affected_legal_asset| crime_type|    crime_subtype| crime_modality

In [15]:
unpivoted_df = idm_df.selectExpr(*unpivoting_columns
                                 , stack_query_expression)

In [16]:
date_df = unpivoted_df.withColumn('info_month_date', get_first_day_of_month_date_udf('year', 'month').cast(DateType()))

In [17]:
date_df.show()

24/03/26 01:56:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: A�o, Clave_Ent, Entidad, Cve. Municipio, Municipio, Bien jur�dico afectado, Tipo de delito, Subtipo de delito, Modalidad, Enero, Febrero, Marzo, Abril, Mayo, Junio, Julio, Agosto, Septiembre, Octubre, Noviembre, Diciembre
 Schema: year, entity_code, entity_name, municipality_code, municipality_name, affected_legal_asset, crime_type, crime_subtype, crime_modality_type, january, february, march, april, may, june, july, august, september, october, november, december
Expected: year but found: A�o
CSV file: gs://landing_bucket_dez/idm/idm.csv
+----+-----------+--------------+-----------------+-----------------+--------------------+----------+----------------+-------------------+---------+------+---------------+
|year|entity_code|   entity_name|municipality_code|municipality_name|affected_legal_asset|crime_type|   crime_subtype|crime_modality_type|    month|crimes|info_month_date|
+----+-----------+-

In [25]:
date_df.write\
.mode("overwrite")\
.partitionBy('year')\
.parquet('gs://landing_bucket_dez/pq/idm/')

24/03/17 00:14:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: A�o, Clave_Ent, Entidad, Cve. Municipio, Municipio, Bien jur�dico afectado, Tipo de delito, Subtipo de delito, Modalidad, Enero, Febrero, Marzo, Abril, Mayo, Junio, Julio, Agosto, Septiembre, Octubre, Noviembre, Diciembre
 Schema: year, entity_code, entity_name, municipality_code, municipality_name, affected_legal_asset, crime_type, crime_subtype, crime_modality_type, january, february, march, april, may, june, july, august, september, october, november, december
Expected: year but found: A�o
CSV file: gs://landing_bucket_dez/idm.csv
